In [19]:
import os
import openai
import pandas
from Bio import Entrez
import time

In [ ]:
#juan
%env YOUR_ORG_ID=org-TpGuUg********************

%env OPENAI_API_KEY= sk-RDEji********************

openai.api_key = "sk-RDEji********************"

# list models
models = openai.Model.list()

model_select = "gpt-3.5-turbo" #OR gpt-4-0314

In [21]:
from pandas import read_csv

filename = "address-to-input-csv-file"

data = read_csv(filename)
print(data.shape)


(100, 6)


In [22]:
data['joined_content']="NA"
data['Note']="NA"

data['Chat_species']="NA"
data['token_species']="NA"
data['FoundYES_species']="NA"
data['FoundNO_species']="NA"
data['FoundNOTSURE_species']="NA"
data['FoundHOWEVER_species']="NA"

data['Chat_disease']="NA"
data['token_disease']="NA"
data['FoundYES_disease']="NA"
data['FoundNO_disease']="NA"
data['FoundNOTSURE_disease']="NA"
data['FoundHOWEVER_disease']="NA"

data['Chat_treatment']="NA"
data['token_treatment']="NA"
data['FoundYES_treatment']="NA"
data['FoundNO_treatment']="NA"
data['FoundNOTSURE_treatment']="NA"
data['FoundHOWEVER_treatment']="NA"

data['Chat_research_type']="NA"
data['token_research_type']="NA"
data['FoundYES_research_type']="NA"
data['FoundNO_research_type']="NA"
data['FoundNOTSURE_research_type']="NA"
data['FoundHOWEVER_research_type']="NA"

data['AllYES_summary']="NA"

In [23]:
outputpath='address-to-output-csv-file'

In [25]:
na_rows = data[data['AllYES_summary']=='NA'].index.tolist()

In [ ]:
print(na_rows)

In [ ]:
for i in na_rows:
    try:
        print(i)
        temp_Title=data["Title"][i]
        temp_PMID=data["PMID"][i]
        #print(temp_PMID)
        #print(temp_Title)
        # Define the PMID of the article you want to retrieve
        pmid = temp_PMID

        # Use Entrez.efetch() to retrieve the article summary
        pmid_str = str(pmid)


        # Extract the article abstract from the record
        try:
            handle = Entrez.efetch(db='pubmed', id=pmid_str, retmode='xml')
            record = Entrez.read(handle)
            abstract = record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText']
        except KeyError:
            abstract = 'No abstract found'

        temp_Abstract=" ".join(abstract)

        #print(temp_Abstract)
        joined_content=''.join(["Title: ",temp_Title,". Abstract: ",temp_Abstract])

        if len(joined_content) < 3000:
            data["joined_content"][i] = joined_content
            data["Note"][i] = "qualified length"
        else:
            data["joined_content"][i] = joined_content
            data["Note"][i] = "length is too long"
            joined_content = joined_content[0:3000]

        print(joined_content)

        #species#
        try:
            completion = openai.ChatCompletion.create(
                model=model_select, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant. I will give you title and abstract of a publication and you will reply whether it meets our criteria or not. I want you to only reply with yes, no, or not sure, and followed with reasons. The criteria is: studies that use human as primary research subject."},
                          {"role": "user", "content": joined_content}]
            )
        except KeyError:
            completion = 'completion ERROR'
        time.sleep(0.5)

        print(completion)
        if completion == 'completion ERROR':
            data["Chat_species"][i]='completion ERROR'
            data["token_species"][i]=0
        else:    
            data["Chat_species"][i]=completion.choices[0].message['content']
            data["token_species"][i]=completion.usage.total_tokens

        if "yes" in data["Chat_species"][i][0:3].lower():
            data["FoundYES_species"][i]="Found"
        else:
            data["FoundYES_species"][i]="NotFound"

        if "no." in data["Chat_species"][i][0:3].lower():
            data["FoundNO_species"][i]="Found"
        elif "no," in data["Chat_species"][i][0:3].lower():
            data["FoundNO_species"][i]="Found"
        else:
            data["FoundNO_species"][i]="NotFound"

        if "not sure" in data["Chat_species"][i][0:10].lower():
            data["FoundNOTSURE_species"][i]="Found"
        else:
            data["FoundNOTSURE_species"][i]="NotFound"

        if "however" in data["Chat_species"][i].lower():
            data["FoundHOWEVER_species"][i]="Found"
        else:
            data["FoundHOWEVER_species"][i]="NotFound"

        data.to_csv(outputpath,sep=',',index=True,header=True)
        
        #disease#
        try:
            completion = openai.ChatCompletion.create(
                model=model_select, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant. I will give you title and abstract of a publication and you will reply whether it meets our criteria or not. I want you to only reply with yes, no, or not sure, and followed with reasons. The criteria is: studies that involve patients with glioma, glioblastoma, astrocytoma, oligodendroglioma."},
                          {"role": "user", "content": joined_content}]
            )
        except KeyError:
            completion = 'completion ERROR'
        time.sleep(0.5)

        print(completion)
        if completion == 'completion ERROR':
            data["Chat_disease"][i]='completion ERROR'
            data["token_disease"][i]=0
        else:    
            data["Chat_disease"][i]=completion.choices[0].message['content']
            data["token_disease"][i]=completion.usage.total_tokens

        if "yes" in data["Chat_disease"][i][0:3].lower():
            data["FoundYES_disease"][i]="Found"
        else:
            data["FoundYES_disease"][i]="NotFound"

        if "no." in data["Chat_disease"][i][0:3].lower():
            data["FoundNO_disease"][i]="Found"
        elif "no," in data["Chat_disease"][i][0:3].lower():
            data["FoundNO_disease"][i]="Found"
        else:
            data["FoundNO_disease"][i]="NotFound"

        if "not sure" in data["Chat_disease"][i][0:10].lower():
            data["FoundNOTSURE_disease"][i]="Found"
        else:
            data["FoundNOTSURE_disease"][i]="NotFound"

        if "however" in data["Chat_disease"][i].lower():
            data["FoundHOWEVER_disease"][i]="Found"
        else:
            data["FoundHOWEVER_disease"][i]="NotFound"

        data.to_csv(outputpath,sep=',',index=True,header=True)
        
        #treatment#
        try:
            completion = openai.ChatCompletion.create(
                model=model_select, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant. I will give you title and abstract of a publication and you will reply whether it meets our criteria or not. I want you to only reply with yes, no, or not sure, and followed with reasons. The criteria is: The title and abstract must explicitly mention that the study involves treatment of patients with one or more of the following immunotherapies: Immune Checkpoint Inhibitors (e.g. anti-PD-1, anti-PD-L1, anti-CTLA-4), Peptide Vaccination, Dendritic cell Vaccination, Adoptive Transfer of Effector Lymphocytes (e.g. CAR-T cell therapy, TCR-engineered T cells), Chimeric Antigen Receptor (CAR) T-cell therapy, or Oncolytic Viral Vectors therapy."},
                          {"role": "user", "content": joined_content}]
            )
        except KeyError:
            completion = 'completion ERROR'
        time.sleep(0.5)

        print(completion)
        if completion == 'completion ERROR':
            data["Chat_treatment"][i]='completion ERROR'
            data["token_treatment"][i]=0
        else:    
            data["Chat_treatment"][i]=completion.choices[0].message['content']
            data["token_treatment"][i]=completion.usage.total_tokens

        if "yes" in data["Chat_treatment"][i][0:3].lower():
            data["FoundYES_treatment"][i]="Found"
        else:
            data["FoundYES_treatment"][i]="NotFound"

        if "no." in data["Chat_treatment"][i][0:3].lower():
            data["FoundNO_treatment"][i]="Found"
        elif "no," in data["Chat_treatment"][i][0:3].lower():
            data["FoundNO_treatment"][i]="Found"
        else:
            data["FoundNO_treatment"][i]="NotFound"

        if "not sure" in data["Chat_treatment"][i][0:10].lower():
            data["FoundNOTSURE_treatment"][i]="Found"
        else:
            data["FoundNOTSURE_treatment"][i]="NotFound"

        if "however" in data["Chat_treatment"][i].lower():
            data["FoundHOWEVER_treatment"][i]="Found"
        else:
            data["FoundHOWEVER_treatment"][i]="NotFound"

        data.to_csv(outputpath,sep=',',index=True,header=True)
        
        #research_type#
        try:
            completion = openai.ChatCompletion.create(
                model=model_select, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant. I will give you title and abstract of a publication and you will reply whether it meets our criteria or not. I want you to only reply with yes, no, or not sure, and followed with reasons. The criteria is: studies that was original research instead of reviews, protocol, or case report with less than five patients."},
                          {"role": "user", "content": joined_content}]
            )
        except KeyError:
            completion = 'completion ERROR'
        time.sleep(0.5)

        print(completion)
        if completion == 'completion ERROR':
            data["Chat_research_type"][i]='completion ERROR'
            data["token_research_type"][i]=0
        else:    
            data["Chat_research_type"][i]=completion.choices[0].message['content']
            data["token_research_type"][i]=completion.usage.total_tokens

        if "yes" in data["Chat_research_type"][i][0:3].lower():
            data["FoundYES_research_type"][i]="Found"
        else:
            data["FoundYES_research_type"][i]="NotFound"

        if "no." in data["Chat_research_type"][i][0:3].lower():
            data["FoundNO_research_type"][i]="Found"
        elif "no," in data["Chat_research_type"][i][0:3].lower():
            data["FoundNO_research_type"][i]="Found"
        else:
            data["FoundNO_research_type"][i]="NotFound"

        if "not sure" in data["Chat_research_type"][i][0:10].lower():
            data["FoundNOTSURE_research_type"][i]="Found"
        else:
            data["FoundNOTSURE_research_type"][i]="NotFound"

        if "however" in data["Chat_research_type"][i].lower():
            data["FoundHOWEVER_research_type"][i]="Found"
        else:
            data["FoundHOWEVER_research_type"][i]="NotFound"
        
        if data["FoundYES_species"][i]=="Found" and data["FoundYES_disease"][i]=="Found" and data["FoundYES_treatment"][i]=="Found" and data["FoundYES_research_type"][i]=="Found":
            data["AllYES_summary"][i]="Yes"
        else: 
            data["AllYES_summary"][i]="No"
            
        data.to_csv(outputpath,sep=',',index=True,header=True)
    except:
        pass